# Lecture 3, Lab

Submit via link with [Gradescope](http://gradescope.com)

# Looking at BigQuery Results

Go ahead and open up a BigQuery console with [Google Compute Platform](https://console.cloud.google.com/welcome?project=neu-cs6220). Let's review a [BigQuery Table that is a real dataset from Google](https://console.cloud.google.com/bigquery?sq=568122555620:c3135eff33c041a78b04d293104b04da). This dataset is populated daily and is processed from Google Search logs. To do this, you will need to:

1. Go to [this link](https://console.cloud.google.com/bigquery?sq=568122555620:c3135eff33c041a78b04d293104b04da)
1. When prompted, create a `project`
1. Run the SQL query
1. Click on `Explore Data` on the lower right corner
1. Then, click on `Explore in Python Notebook`

You can go ahead and explore the data!

# Setting Up Spark in Colabs

## Virtual Machine Setup

Colab is an interactive virtual machine that Google provides for free. You could alternatively work locally on Jupyter Notebooks or Python scripts so that you can customize your environment, needn't setup Spark, and not worry about constantly installing packages. Otherwise, the below can be done in class.

The below code sets up your Colab notebook. (You could do the same in Jupyter or in your shell on your local machine.) If you executed the cells below correctly, you should be able to see the file *pg100.txt* under the "Files" tab on the left panel.

In [1]:
#@title Download and install the PySpark packages

!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=6c1afaea15a50ddf92dc4697485b84ba824ac976d4556d3fb7fab986ac7e2014
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [7]:
#@title Setup Spark and Data

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

data = sc.textFile("soc-LiveJournal1Adj.txt")

adj_list = data.map(lambda line: line.split("\t")) \
                   .filter(lambda line: len(line) == 2 and line[0].isdigit() and line[1].strip() != "") \
                   .map(lambda line: (int(line[0]), list(set(map(int, line[1].split(","))))))

adj_list_dict = adj_list.collectAsMap()

adj_list_broadcast = sc.broadcast(adj_list_dict)

'''
  Generate a defined relation pair (U, (X, Y)) for a given user id(U).
  Here user U and user X are mutual friends, user X and user Y are mutual friends, but user U and user Y are not mutual friends.
  For example, (1, (0, 2)) means user 1 and user 0 are mutual friends, user 2 and user 0 are mutual friends, but user 1 and user 2 are not mutual friends.
'''
def mutual_friends_pairs(user_id):
    pairs = []
    user_friends = adj_list_broadcast.value.get(user_id, [])
    for user_friend in user_friends:
        mutual_friends = adj_list_broadcast.value.get(user_friend, [])
        for mutual_friend in mutual_friends:
            if mutual_friend != user_id and mutual_friend not in user_friends:
                pairs.append((user_id, (user_friend, mutual_friend)))
    return pairs

friend_pairs_rdd = adj_list.flatMap(lambda x: mutual_friends_pairs(x[0]))

# friend_pairs_rdd = friend_pairs_rdd.filter(lambda pair: pair[0] == 11 or pair[0] == 924)

# Step 1
recommand_count_rdd = friend_pairs_rdd.map(lambda pair: (pair[0], pair[1][1])) \
                                      .map(lambda user_mutual: ((user_mutual[0], user_mutual[1]), 1)) \
                                      .reduceByKey(lambda a, b: a + b)

# Step 2: (user_id, mutual_friend) => count => [(mutual_friend, count), ...]
user_recommendations_rdd = recommand_count_rdd.map(lambda x: (x[0][0], (x[0][1], x[1]))) \
                                              .groupByKey() \
                                              .mapValues(list)

# Step 3
sorted_recommendations_rdd = user_recommendations_rdd.mapValues(lambda recommendations: sorted(recommendations, key=lambda x: (-x[1], x[0]))[:10])

# Step 4
top_recommendations_rdd = sorted_recommendations_rdd.mapValues(lambda recommendations: [x[0] for x in recommendations])

top_recommendations_rdd.map(lambda x: f"{x[0]}\t{x[1]}").coalesce(1).saveAsTextFile("output.txt")